In [1]:
pip install selenium googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=f8df50ffb3894e7d29b61bd296ce738ea4c55ae91839f14115f5514d1dc23301
  Stored in directory: /Users/joyeongyeong/Library/Caches/pip/wheels/95/0f/04/b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Unin

In [15]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from googletrans import Translator

# ChromeDriver 설정
def get_driver():
    options = Options()
    options.add_argument("--headless")  # 브라우저 창 숨김
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    
    # chromedriver의 절대 경로로 수정
    service = Service(executable_path="/Users/joyeongyeong/Documents/Text_Analysis/Assignment2/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    return driver


# 리뷰 데이터 스크래핑
def scrape_reviews(url, driver, max_reviews=500):
    driver.get(url)
    time.sleep(3)
    reviews = []
    
    while len(reviews) < max_reviews:
        try:
            review_elements = driver.find_elements(By.CSS_SELECTOR, ".review-text-content span")
            for review in review_elements:
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())
            
            # 다음 페이지로 이동
            next_button = driver.find_element(By.CSS_SELECTOR, ".a-pagination .a-last a")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(3)
            else:
                break
        except Exception as e:
            print(f"에러 발생: {e}")
            break
    
    return reviews

# 리뷰 번역
def translate_reviews(reviews, target_lang="en"):
    translator = Translator()
    translated_reviews = []
    for review in reviews:
        try:
            translated = translator.translate(review, dest=target_lang)
            translated_reviews.append(translated.text)
        except Exception as e:
            print(f"번역 에러: {e}")
            translated_reviews.append("번역 실패")
    return translated_reviews

# CSV 저장
def save_to_csv(data, filename="amazon_reviews.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Original Review", "Translated Review"])
        writer.writerows(data)

# 메인 실행
if __name__ == "__main__":
    amazon_url = "https://www.amazon.fr/-/en/dp/B00ST2GSRK/?_encoding=UTF8&pd_rd_w=lpfQa&content-id=amzn1.sym.90eb7b98-a9ec-4276-971f-11669ff2eae8%3Aamzn1.symc.9b8fba90-e74e-4690-b98f-edc36fe735a6&pf_rd_p=90eb7b98-a9ec-4276-971f-11669ff2eae8&pf_rd_r=AZT9GXHARTV8QCTNS2N4&pd_rd_wg=C1EXc&pd_rd_r=7dba938e-ec93-418f-8034-898d1f5b3b73&ref_=pd_hp_d_btf_ci_mcx_mr_ca_id_hp_d#customerReviews"
    driver = get_driver()
    try:
        print("리뷰 스크래핑 시작...")
        reviews = scrape_reviews(amazon_url, driver)
        print(f"{len(reviews)}개의 리뷰 스크래핑 완료.")
        
        print("리뷰 번역 시작...")
        translated_reviews = translate_reviews(reviews)
        
        print("CSV 저장 시작...")
        save_to_csv(zip(reviews, translated_reviews))
        print("작업 완료: amazon_reviews.csv")
    finally:
        driver.quit()


리뷰 스크래핑 시작...
에러 발생: Message: no such element: Unable to locate element: {"method":"css selector","selector":".a-pagination .a-last a"}
  (Session info: chrome=132.0.6834.84); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102c7f7a4 cxxbridge1$str$ptr + 2589716
1   chromedriver                        0x0000000102c7805c cxxbridge1$str$ptr + 2559180
2   chromedriver                        0x000000010281bf5c cxxbridge1$string$len + 88260
3   chromedriver                        0x0000000102861024 cxxbridge1$string$len + 371084
4   chromedriver                        0x000000010289b1a8 cxxbridge1$string$len + 609040
5   chromedriver                        0x0000000102854e40 cxxbridge1$string$len + 321448
6   chromedriver                        0x0000000102855a88 cxxbridge1$string$len + 324592
7   chromedriver                        0x

In [23]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from googletrans import Translator


# ChromeDriver 설정
def get_driver():
    options = Options()
    options.add_argument("--headless")  # 브라우저 창 숨김
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    service = Service(executable_path="/Users/joyeongyeong/Documents/Text_Analysis/Assignment2/chromedriver")  # chromedriver 절대 경로 설정
    driver = webdriver.Chrome(service=service, options=options)
    return driver


# 리뷰 데이터 스크래핑
def scrape_reviews(url, driver, max_reviews=500):
    driver.get(url)
    time.sleep(3)  # 페이지 로드 대기
    reviews = []
    page = 1

    while len(reviews) < max_reviews:
        try:
            print(f"페이지 {page} 리뷰 스크래핑 중...")
            
            # 모든 리뷰 요소가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".review-text-content span"))
            )
            
            # 현재 페이지 리뷰 가져오기
            review_elements = driver.find_elements(By.CSS_SELECTOR, ".review-text-content span")
            for review in review_elements:
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())
            
            # 다음 페이지로 이동
            next_button = driver.find_element(By.CSS_SELECTOR, ".a-pagination .a-last a")
            if next_button.is_enabled():
                next_button.click()
                page += 1
                time.sleep(3)  # 다음 페이지 로드 대기
            else:
                print("다음 페이지 버튼을 찾을 수 없습니다. 종료합니다.")
                break
        except Exception as e:
            print(f"에러 발생: {e}")
            break

    return reviews


# 리뷰 번역
def translate_reviews(reviews, target_lang="en"):
    translator = Translator()
    translated_reviews = []
    for review in reviews:
        try:
            translated = translator.translate(review, dest=target_lang)
            translated_reviews.append(translated.text)
        except Exception as e:
            print(f"번역 에러: {e}")
            translated_reviews.append("번역 실패")
    return translated_reviews


# CSV 저장
def save_to_csv(data, filename="amazon_reviews_2.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Original Review", "Translated Review"])
        writer.writerows(data)


# 메인 실행
if __name__ == "__main__":
    amazon_url = "https://www.amazon.fr/-/en/product-reviews/B00ST2GSRK/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
    driver = get_driver()
    try:
        print("리뷰 스크래핑 시작...")
        reviews = scrape_reviews(amazon_url, driver, max_reviews=500)
        print(f"{len(reviews)}개의 리뷰 스크래핑 완료.")
        
        print("리뷰 번역 시작...")
        translated_reviews = translate_reviews(reviews)
        
        print("CSV 저장 시작...")
        save_to_csv(zip(reviews, translated_reviews))
        print("작업 완료: amazon_reviews_2.csv")
    finally:
        driver.quit()


리뷰 스크래핑 시작...
페이지 1 리뷰 스크래핑 중...
에러 발생: Message: 

0개의 리뷰 스크래핑 완료.
리뷰 번역 시작...
CSV 저장 시작...
작업 완료: amazon_reviews_2.csv


In [25]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from googletrans import Translator


# ChromeDriver 설정
def get_driver():
    options = Options()
    options.add_argument("--headless")  # 브라우저 창 숨김
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    service = Service(executable_path="/Users/joyeongyeong/Documents/Text_Analysis/Assignment2/chromedriver")  # chromedriver 경로 설정
    driver = webdriver.Chrome(service=service, options=options)
    return driver


# 리뷰 데이터 스크래핑
def scrape_reviews(url, driver, max_reviews=500):
    driver.get(url)
    time.sleep(3)  # 초기 페이지 로드 대기
    reviews = []
    page = 1

    while len(reviews) < max_reviews:
        try:
            print(f"페이지 {page} 리뷰 스크래핑 중...")
            
            # 현재 페이지에서 리뷰 가져오기
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".review-text-content span"))
            )
            review_elements = driver.find_elements(By.CSS_SELECTOR, ".review-text-content span")
            for review in review_elements:
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())
            
            # 다음 페이지 버튼 확인 및 클릭
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, ".a-pagination .a-last a")
                if next_button.is_enabled():
                    next_button.click()
                    page += 1
                    time.sleep(3)  # 다음 페이지 로드 대기
                else:
                    print("더 이상 페이지가 없습니다.")
                    break
            except Exception:
                print("다음 페이지 버튼을 찾을 수 없습니다.")
                break

        except Exception as e:
            print(f"에러 발생: {e}")
            break

    return reviews


# 리뷰 번역
def translate_reviews(reviews, target_lang="en"):
    translator = Translator()
    translated_reviews = []
    for review in reviews:
        try:
            translated = translator.translate(review, dest=target_lang)
            translated_reviews.append(translated.text)
        except Exception as e:
            print(f"번역 에러: {e}")
            translated_reviews.append("번역 실패")
    return translated_reviews


# CSV 저장
def save_to_csv(data, filename="amazon_reviews_3.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Original Review", "Translated Review"])
        writer.writerows(data)


# 메인 실행
if __name__ == "__main__":
    amazon_url = "https://www.amazon.fr/-/en/product-reviews/B00ST2GSRK/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2"
    driver = get_driver()
    try:
        print("리뷰 스크래핑 시작...")
        reviews = scrape_reviews(amazon_url, driver, max_reviews=500)
        print(f"{len(reviews)}개의 리뷰 스크래핑 완료.")
        
        print("리뷰 번역 시작...")
        translated_reviews = translate_reviews(reviews)
        
        print("CSV 저장 시작...")
        save_to_csv(zip(reviews, translated_reviews))
        print("작업 완료: amazon_reviews_3.csv")
    finally:
        driver.quit()


리뷰 스크래핑 시작...
페이지 1 리뷰 스크래핑 중...
에러 발생: Message: 

0개의 리뷰 스크래핑 완료.
리뷰 번역 시작...
CSV 저장 시작...
작업 완료: amazon_reviews_3.csv


In [27]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from googletrans import Translator

# ChromeDriver 설정
def get_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    service = Service(executable_path="/Users/joyeongyeong/Documents/Text_Analysis/Assignment2/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# 리뷰 스크래핑
def scrape_reviews(url, driver, max_reviews=500):
    driver.get(url)
    time.sleep(3)
    reviews = []
    page = 1

    while len(reviews) < max_reviews:
        try:
            print(f"페이지 {page} 리뷰 스크래핑 중...")

            # 스크롤 내려 모든 리뷰 로드
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

            # 리뷰 요소 가져오기
            review_elements = driver.find_elements(By.CSS_SELECTOR, ".a-size-base.review-text.review-text-content span")
            for review in review_elements:
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())

            # 다음 페이지 버튼 클릭
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".a-pagination .a-last a"))
                )
                next_button.click()
                page += 1
                time.sleep(3)
            except Exception:
                print("다음 페이지 버튼을 찾거나 클릭할 수 없습니다.")
                break

        except Exception as e:
            print(f"에러 발생: {e}")
            break

    return reviews

# 리뷰 번역
def translate_reviews(reviews, target_lang="en"):
    translator = Translator()
    translated_reviews = []
    for review in reviews:
        try:
            translated = translator.translate(review, dest=target_lang)
            translated_reviews.append(translated.text)
        except Exception as e:
            print(f"번역 에러: {e}")
            translated_reviews.append("번역 실패")
    return translated_reviews

# CSV 저장
def save_to_csv(data, filename="amazon_reviews_fixed.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Original Review", "Translated Review"])
        writer.writerows(data)

# 메인 실행
if __name__ == "__main__":
    amazon_url = "https://www.amazon.fr/product-reviews/B00ST2GSRK"
    driver = get_driver()
    try:
        print("리뷰 스크래핑 시작...")
        reviews = scrape_reviews(amazon_url, driver, max_reviews=500)
        print(f"{len(reviews)}개의 리뷰 스크래핑 완료.")

        print("리뷰 번역 시작...")
        translated_reviews = translate_reviews(reviews)

        print("CSV 저장 시작...")
        save_to_csv(zip(reviews, translated_reviews))
        print("작업 완료: amazon_reviews_fixed.csv")
    finally:
        driver.quit()


리뷰 스크래핑 시작...
페이지 1 리뷰 스크래핑 중...
다음 페이지 버튼을 찾거나 클릭할 수 없습니다.
0개의 리뷰 스크래핑 완료.
리뷰 번역 시작...
CSV 저장 시작...
작업 완료: amazon_reviews_fixed.csv


In [29]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from googletrans import Translator
from random import randint

# ChromeDriver 설정
def get_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    service = Service(executable_path="/Users/joyeongyeong/Documents/Text_Analysis/Assignment2/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# CAPTCHA 감지
def detect_captcha(driver):
    try:
        page_source = driver.page_source
        if "robot" in page_source.lower() or "captcha" in page_source.lower():
            print("CAPTCHA가 감지되었습니다. 수동 확인 필요.")
            return True
        return False
    except Exception as e:
        print(f"CAPTCHA 감지 중 에러 발생: {e}")
        return False

# 리뷰 데이터 스크래핑
def scrape_reviews(url, driver, max_reviews=500):
    driver.get(url)
    time.sleep(3)  # 초기 페이지 로드 대기
    reviews = []
    page = 1

    while len(reviews) < max_reviews:
        try:
            if detect_captcha(driver):
                break  # CAPTCHA 발생 시 중단

            print(f"페이지 {page} 리뷰 스크래핑 중...")

            # 스크롤 내려 모든 리뷰 로드
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(randint(3, 6))  # 요청 간 랜덤 지연

            # 리뷰 요소 가져오기 (CSS Selector 유연성 적용)
            selectors = [
                ".a-size-base.review-text.review-text-content span",  # 기본 Selector
                ".review-text-content span",                         # 대체 Selector
            ]
            review_elements = []
            for selector in selectors:
                try:
                    review_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    if review_elements:
                        break  # 리뷰가 있으면 멈춤
                except Exception:
                    continue
            
            # 리뷰 저장
            for review in review_elements:
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())

            # 다음 페이지 버튼 클릭
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".a-pagination .a-last a"))
                )
                next_button.click()
                page += 1
                time.sleep(randint(3, 6))  # 요청 간 랜덤 지연
            except Exception:
                print("다음 페이지 버튼을 찾거나 클릭할 수 없습니다.")
                break

        except Exception as e:
            print(f"에러 발생: {e}")
            break

    return reviews

# 리뷰 번역
def translate_reviews(reviews, target_lang="en"):
    translator = Translator()
    translated_reviews = []
    for review in reviews:
        try:
            translated = translator.translate(review, dest=target_lang)
            translated_reviews.append(translated.text)
        except Exception as e:
            print(f"번역 에러: {e}")
            translated_reviews.append("번역 실패")
    return translated_reviews

# CSV 저장
def save_to_csv(data, filename="amazon_reviews_improved.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Original Review", "Translated Review"])
        writer.writerows(data)

# 메인 실행
if __name__ == "__main__":
    amazon_url = "https://www.amazon.fr/product-reviews/B00ST2GSRK"
    driver = get_driver()
    try:
        print("리뷰 스크래핑 시작...")
        reviews = scrape_reviews(amazon_url, driver, max_reviews=500)
        print(f"{len(reviews)}개의 리뷰 스크래핑 완료.")

        print("리뷰 번역 시작...")
        translated_reviews = translate_reviews(reviews)

        print("CSV 저장 시작...")
        save_to_csv(zip(reviews, translated_reviews))
        print("작업 완료: amazon_reviews_improved.csv")
    finally:
        driver.quit()


리뷰 스크래핑 시작...
CAPTCHA가 감지되었습니다. 수동 확인 필요.
0개의 리뷰 스크래핑 완료.
리뷰 번역 시작...
CSV 저장 시작...
작업 완료: amazon_reviews_improved.csv


In [31]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    service = Service(executable_path="/Users/joyeongyeong/Documents/Text_Analysis/Assignment2/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def scrape_reviews(url, driver, max_reviews=500):
    driver.get(url)
    time.sleep(3)

    # 페이지 소스 확인
    print("페이지 소스를 확인합니다...")
    print(driver.page_source)  # HTML 출력
    
    reviews = []
    page = 1

    while len(reviews) < max_reviews:
        try:
            print(f"페이지 {page} 리뷰 스크래핑 중...")

            # 페이지 끝까지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            # 리뷰 요소 가져오기
            try:
                review_elements = driver.find_elements(By.CSS_SELECTOR, ".a-size-base.review-text.review-text-content span")
                print(f"찾은 리뷰 수: {len(review_elements)}")  # 리뷰 요소 수 확인
            except Exception as e:
                print(f"리뷰 요소 탐색 실패: {e}")
                break

            for review in review_elements:
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())

            # 다음 페이지로 이동
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".a-pagination .a-last a"))
                )
                next_button.click()
                page += 1
                time.sleep(3)
            except Exception:
                print("다음 페이지 버튼을 찾을 수 없습니다.")
                break

        except Exception as e:
            print(f"에러 발생: {e}")
            break

    return reviews

if __name__ == "__main__":
    amazon_url = "https://www.amazon.fr/product-reviews/B00ST2GSRK"
    driver = get_driver()
    try:
        print("리뷰 스크래핑 시작...")
        reviews = scrape_reviews(amazon_url, driver, max_reviews=500)
        print(f"스크래핑된 리뷰 수: {len(reviews)}")
    finally:
        driver.quit()


리뷰 스크래핑 시작...
페이지 소스를 확인합니다...
<html class=" a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-hires a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.24.10-2025-01-06"><head><script async="" src="https://images-eu.ssl-images-amazon.com/images/I/31bJewCvY-L.js" crossorigin="anonymous"></script>
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d()

In [33]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    service = Service(executable_path="/Users/joyeongyeong/Documents/Text_Analysis/Assignment2/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def scrape_reviews(url, driver, max_reviews=500):
    driver.get(url)
    time.sleep(3)

    # 페이지 소스 확인 (HTML 구조 확인용)
    print("페이지 소스를 확인합니다...")
    print(driver.page_source)
    
    reviews = []
    page = 1

    while len(reviews) < max_reviews:
        try:
            print(f"페이지 {page} 리뷰 스크래핑 중...")

            # 페이지 끝까지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            # 리뷰 요소 가져오기 (여러 CSS Selector 시도)
            selectors = [
                ".a-size-base.review-text-content span",  # 가장 일반적인 리뷰 Selector
                ".review-text-content span",             # 대체 Selector
                ".a-size-base.review-text"               # 추가 Selector
            ]

            review_elements = []
            for selector in selectors:
                try:
                    review_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    if review_elements:  # 리뷰 요소를 찾으면 멈춤
                        break
                except Exception as e:
                    print(f"Selector {selector}로 실패: {e}")
                    continue

            print(f"찾은 리뷰 수: {len(review_elements)}")  # 리뷰 요소 수 확인

            for review in review_elements:
                if len(reviews) >= max_reviews:
                    break
                reviews.append(review.text.strip())

            # 다음 페이지 버튼 클릭
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".a-pagination .a-last a"))
                )
                next_button.click()
                page += 1
                time.sleep(3)
            except Exception:
                print("다음 페이지 버튼을 찾을 수 없습니다.")
                break

        except Exception as e:
            print(f"에러 발생: {e}")
            break

    return reviews

if __name__ == "__main__":
    amazon_url = "https://www.amazon.fr/-/en/product-reviews/B00ST2GSRK/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1"
    driver = get_driver()
    try:
        print("리뷰 스크래핑 시작...")
        reviews = scrape_reviews(amazon_url, driver, max_reviews=500)
        print(f"스크래핑된 리뷰 수: {len(reviews)}")
        for i, review in enumerate(reviews[:10]):  # 처음 10개 리뷰 출력
            print(f"{i + 1}: {review}")
    finally:
        driver.quit()


리뷰 스크래핑 시작...
페이지 소스를 확인합니다...
<html class=" a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-hires a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.24.10-2025-01-06"><head><script async="" src="https://images-eu.ssl-images-amazon.com/images/I/31bJewCvY-L.js" crossorigin="anonymous"></script>
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d()